In [1]:
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, FalseNegatives, FalsePositives, TrueNegatives, \
    TruePositives, AUC
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

2023-06-23 22:55:32.831372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 22:55:33.316563: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14340628342779192628
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5919604736
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14870977151254056121
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:2b:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-06-23 22:55:35.072789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 22:55:35.086646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 22:55:35.086777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-06-23 22:55:35.609588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 22:55:35.609748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 22:55:35.609850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# Set the GPU device
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

In [5]:
# Set GPU device
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Replace '0' with the GPU device ID

In [6]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 8

img_data_gn = ImageDataGenerator(rescale=1./255)

In [7]:
train_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/train"
valid_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/validation"

In [8]:
train_batches = img_data_gn.flow_from_directory(train_path,
                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                class_mode='binary',
                                                classes=['authentic', 'tampered'],
                                                batch_size=BATCH_SIZE,
                                                shuffle=True)

valid_batches = img_data_gn.flow_from_directory(valid_path,
                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                class_mode='binary',
                                                classes=['authentic', 'tampered'],
                                                batch_size=BATCH_SIZE, shuffle=True)

Found 20800 images belonging to 2 classes.
Found 8443 images belonging to 2 classes.


In [9]:
train_batches.class_indices

{'authentic': 0, 'tampered': 1}

In [10]:
model = ResNet152V2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights= None , include_top=False)


2023-06-23 22:55:36.163546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 22:55:36.163694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 22:55:36.163795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
# for layer in model.layers:
#     layer.trainable = False
model.trainable = False

In [12]:
last_layer = model.get_layer('conv5_block3_out')
last_output = last_layer.output

In [13]:
# x = layers.Flatten()(last_output)
# x = layers.Dense(1000, activation='relu')(x)
# x = layers.Dense(1000, activation='relu')(x)
# x = layers.Dense(1, activation='sigmoid')(x)

In [14]:
x = layers.Flatten()(last_output)
x = layers.Dense(1000, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = layers.Dense(1000, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(500, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x) 
x = Dense(500, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x) 
x = layers.Dense(1, activation='sigmoid')(x)

In [15]:
newModel = Model(model.input, x)

In [16]:
newModel.compile(Adam(learning_rate=0.001), loss=BinaryCrossentropy(),
                 metrics=[AUC(),BinaryAccuracy(), Precision(), Recall(), FalseNegatives(),
                          FalsePositives(), TrueNegatives(), TruePositives(), AUC()])

In [17]:
newModel.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [18]:
# file_path = 'Experiment_8_2.h5'
# callbacks = [
#     ModelCheckpoint(file_path, monitor='val_loss', mode='min', save_best_only=True, verbose=1),
# ]

In [19]:
# newModel.load_weights(file_path)

In [20]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0000001, verbose=1)

In [21]:
#class_weights_dict = {0: 1, 1: 3} 
history1 = newModel.fit(train_batches, verbose=1, validation_data=valid_batches,
                         epochs=25, steps_per_epoch=len(train_batches), validation_steps=len(valid_batches),callbacks=[early_stopping, reduce_lr])

Epoch 1/25


2023-06-23 22:55:39.785137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-23 22:55:50.333297: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-23 22:55:55.034127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-23 22:55:55.931357: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x34800460 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-23 22:55:55.931377: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2023-06-23 22:55:56.107388

2600/2600 [==============================] - ETA: 0s - loss: 0.7838 - auc: 0.5195 - binary_accuracy: 0.5158 - precision: 0.5023 - recall: 0.4827 - false_negatives: 5234.0000 - false_positives: 4838.0000 - true_negatives: 5845.0000 - true_positives: 4883.0000 - auc_1: 0.5195

2023-06-23 23:05:50.993405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2600/2600 [==============================] - 842s 317ms/step - loss: 0.7838 - auc: 0.5195 - binary_accuracy: 0.5158 - precision: 0.5023 - recall: 0.4827 - false_negatives: 5234.0000 - false_positives: 4838.0000 - true_negatives: 5845.0000 - true_positives: 4883.0000 - auc_1: 0.5195 - val_loss: 0.6817 - val_auc: 0.5899 - val_binary_accuracy: 0.5570 - val_precision: 0.5584 - val_recall: 0.7519 - val_false_negatives: 1101.0000 - val_false_positives: 2639.0000 - val_true_negatives: 1366.0000 - val_true_positives: 3337.0000 - val_auc_1: 0.5899 - lr: 0.0010
Epoch 2/25
2600/2600 [==============================] - 538s 207ms/step - loss: 0.7102 - auc: 0.5468 - binary_accuracy: 0.5366 - precision: 0.5248 - recall: 0.5002 - false_negatives: 5056.0000 - false_positives: 4583.0000 - true_negatives: 6100.0000 - true_positives: 5061.0000 - auc_1: 0.5468 - val_loss: 0.6854 - val_auc: 0.5665 - val_binary_accuracy: 0.5477 - val_precision: 0.5588 - val_recall: 0.6625 - val_false_negatives: 1498.0000 - v

In [22]:
newModel.save("resnet 152.h5")

In [23]:
test_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/test"
test_batches = img_data_gn.flow_from_directory(test_path,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode='binary',
                                               classes=['authenic', 'tampered'],
                                               batch_size=32)

Found 3639 images belonging to 2 classes.


In [24]:
test_image_count = sum([len(files) for _, _, files in os.walk(test_path)])
test_steps = math.ceil(test_image_count / BATCH_SIZE)


In [25]:
test_loss, test_accuracy = newModel.evaluate(test_batches, steps=test_steps)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

2023-06-23 23:47:46.139964: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


914/914 [==============================] - 149s 123ms/step - loss: 0.8616 - auc: 0.0000e+00 - binary_accuracy: 0.4509 - precision: 1.0000 - recall: 0.4509 - false_negatives: 1998.0000 - false_positives: 0.0000e+00 - true_negatives: 0.0000e+00 - true_positives: 1641.0000 - auc_1: 0.0000e+00


ValueError: too many values to unpack (expected 2)

In [ ]:
model_json = newModel.to_json()


In [ ]:
with open("CNNImageSplicingDetectorModelResNet152_2.5k.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
for steps in range(len(test_batches)):
    test_images, test_labels = next(test_batches)
    (newModel.evaluate(test_images, test_labels))


1/1 [==============================] - 1s 1s/step - loss: 0.7960 - binary_accuracy: 0.7500 - precision: 0.7500 - recall: 1.0000 - false_negatives: 0.0000e+00 - false_positives: 4.0000 - true_negatives: 0.0000e+00 - true_positives: 12.0000 - auc: 0.3854


In [ ]:

#serialize weights to HDF5
newModel.save_weights("resnet 152 weights.h5")
